In [2]:
from io import StringIO

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import NaN
from functions import * # import functions from the .py file

from pylab import rcParams
rcParams["figure.figsize"] = 16, 4

PAYLOADSIZE = 6

if PAYLOADSIZE % 2 != 0:
    print("Alarm! the payload size is not even.")
NUM_16RND = (PAYLOADSIZE-2)//2 # how many 16 bits random number included in each frame
MAX_SEQ = 256 # (decimal) maximum seq number defined by the length of the seq, the length of seq is 1B

ModuleNotFoundError: No module named 'matplotlib'

## Log file

**For a fair comparison between groups, must follow the output format below**
<br>$\color{red}{\text{timestamp_rx }|\text{ len [hex] } \text{ seq [hex] } \text{ payload [hex] }|\text{rssi}}$
<br>$\text{ len [hex]}$ = payload length + 1 (seq. field length [1B])
<br>$\text{ payload [hex]}$ should start with the pseudo sequence number [2B], indicates the starting byte's position in the file.
***
*Example log file output for one frame with reciever timestamp xxxx, frame length, seq number and random payload, received signal strength.*
<br>15:17:18.394 |0f 46 bd f8 f7 ea 41 ac 1b be 05 a0 7f d2 09 d4 |  -91 CRC error
<br>$\color{red}{\text{delimiter for four variables is "|" and delimiter for the payload each byte is the space " ".}}$

In [ ]:
# define the file name
filename = "log"
# import file to jupyter notebook
df = readfile("./" + filename + ".txt")
# check the imported data first 10 lines
df.head(10)

In [ ]:
# delete the corrupted packets (PAYLOADSIZE + 2B pesudo sequence number)
df = df[df.payload.apply(lambda x: len(x)==((PAYLOADSIZE)*3-1))]
df.reset_index(inplace=True)

In [ ]:
# replace the resettable seq number to unresettable, for the purpose of reliability calculation
test = replace_seq(df.copy(), MAX_SEQ=256)
test.rename(columns = {'seq':'old_seq', 'new_seq':'seq'}, inplace = True)

# Time

## Definition

**Evaluation metric**
<br>*File Transmission Time: the total latency that certain amount of Data $F$ is delivered to the destination.*
<br> $File\ Delay = Rx\_timestamp[N] - Rx\_timestamp[0]$
<br> where $N$ is the last packet
***
**Interesting to look at**:
<br>*Timestamp difference*: $\Delta T[i] = Rx\_timestamp[i] - Tx\_timestamp[i]$
1. *Worst-case packet delay*: the delay of the packet that takes the longest time to traverse.
2. *Packet delay variation*: the std of the packet delay.

In [ ]:
# compute the file delay
file_delay = df.time_rx[len(df) - 1] - df.time_rx[0]
# convert the timedelta to s
file_delay_s = np.timedelta64(file_delay, "ms").astype(int) / 1000
print(f"The time it takes to transfer the file is : {file_delay}, which is {file_delay_s} seconds.")

# Reliability

## Definition

**Evaluation metric**
<br>*Quality of data transfer.*
<br>*Bit Reliability*: The ratio of data bits correctly delivered to the data sink.
<br>*Bit Error Rate*: The ratio of data bits incorrectly delivered to the data sink.
$$Bit\ Reliability = 1 - BER = 1 - \frac{Number\ of\ wrong\ bit}{Total\ Number\ of\ bit\ sent}$$
***
**Interesting to look at**:
1. *Expected number of transmission*: the ratio of the number of packets correctly received by the data sink and the number of packets sent by the source.

$$ETX = \frac{Number\ of\ packet\ received}{Number\ of\ packet\ sent}$$

2. *Packet reliability*: the number of packet bit correctly delivered to next hop.

## Statistics

In [ ]:
# compute the BER for all received packets
# return the in total ber for received file, error statistics and correct file content supposed to be transmitted
ber, error, file_content = compute_ber(test, PACKET_LEN=NUM_16RND*2, MAX_SEQ=MAX_SEQ)
bit_reliability = (1-ber)*100
print(f"Bit reliability [%]: {bit_reliability}")

In [ ]:
# payload example for debug purpose
# correct sequence for pesudoseq NO. '0000'
tmp = file_content.loc[int('0000', base=16), 'data']
for i in tmp:
    print("{:02x}".format(i), end=" ")

In [ ]:
# BER for each packet
plt.scatter(error.seq, error.bit_error/(PAYLOADSIZE*8), marker='o', s=6, color='black')
plt.grid()
plt.ylabel('Bit Error Rate [%]', fontsize=16)
plt.xlabel('Seq. Number', fontsize=16)

# Distance

## Definition

**The communication distance of the system.**
<br>$D_{1}$: the distance between carrier<->backscatter_tag[m]
<br>$D_{2}$: the distance between backscatter_tag<->receiver[m]
<br> Distance metric = $D_{1}^2D_{2}^2$
<br>

In [ ]:
# record the distance
dis_carrier_tag = 1
dis_tag_rx = 4

dis_metric = dis_carrier_tag**2*dis_tag_rx**2
print(f"Distance metric is: {dis_metric}")

# Radarplot

**Please pay attention to the unit**
<br> <font color='red'>Time</font>: use second as unit
<br> <font color='red'>Reliability</font>: use the percent as unit
<br> <font color='red'>Distance</font>: use meter as unit
<br>**Metics = [Time(s), Reliability(%), Distance(m)]**
<br> <font color='red'>Always keep the reference in your plot.</font>
<br> Tips: for ploting, time metric uses $\frac{1}{Time}*1000$ as the time metric, the higher the value the better the system performance

In [ ]:
metrics = [file_delay_s, bit_reliability, dis_metric]
radar_plot(metrics)

In [ ]:
x = [0.09,0.36,0.56,0.8]

y2826 = [55.589,7.78,1.95,3.14]
y3028 = [53.205,54.738,18.21,15.82]
y3230 = [100,96.966,3.908,46.4]
y3432 = [98.039,98.97,8.26,54.74]
y3634 = [56.09,15.46,49,56.023]
y3836 = [5.58,9.045,53.135,2.74]
y4036 = [98.927,49.746,8.211,54.085]
y4038 = [96.804,4.85,34.519,8.021]

# plotting the points 
line1, = plt.plot(x,y2826, label="28,26")
line2, = plt.plot(x,y3028, label="30,28")
line3, = plt.plot(x,y3230, label="32,30")
line4, = plt.plot(x,y3432, label="34,32")
line5, = plt.plot(x,y3634, label="36,34")
line6, = plt.plot(x,y3836, label="38,36")
line7, = plt.plot(x,y4036, label="40,36")
line8, = plt.plot(x,y4038, label="40,38")

plt.xticks(np.arange(0,1,0.05))
plt.yticks(np.arange(0,100,5))
plt.legend()
plt.ylim(1,100)
plt.xlim(0,1)  
# naming the x axis
plt.xlabel('Distance metric')
# naming the y axis
plt.ylabel('BER')
plt.legend(title='Dividers') 
# giving a title to my graph
plt.title('Divider evaluation')
  
# function to show the plot
plt.show()

In [ ]:
x = [0.09,0.36,0.8]


y3230 = [4.019,1.29,3.735]
y3432 = [4.462,1.79,2.6]
y4036 = [98.95,41.3,14.19]


# plotting the points 

line1, = plt.plot(x,y3230, label="32,30")
line2, = plt.plot(x,y3432, label="34,32")
line3, = plt.plot(x,y4036, label="40,36")

plt.xticks(np.arange(0,1,0.05))
plt.yticks(np.arange(0,100,5))
plt.legend()
plt.ylim(1,100)
plt.xlim(0,1)  
# naming the x axis
plt.xlabel('Distance metric')
# naming the y axis
plt.ylabel('BER')
plt.legend(title='Dividers') 
# giving a title to my graph
plt.title('Baud rate 89.99Kbps')
  
# function to show the plot
plt.show()

In [ ]:
x = [0.09,0.36,0.8,1.44,2.25,3.23]


y3230 = [44.08,3.3,2.125,2.356,0,0]
y3432 = [59.516,13.908,2.787,3.91,0,0]
y4036 = [5.11,60.125,57.369,2.226,16.331,48.228]


# plotting the points 

line1, = plt.plot(x,y3230, label="32,30")
line2, = plt.plot(x,y3432, label="34,32")
line3, = plt.plot(x,y4036, label="40,36")

plt.xticks(np.arange(0,4,0.1))
plt.yticks(np.arange(0,100,5))
plt.legend()
plt.ylim(1,100)
plt.xlim(0,5)  
# naming the x axis
plt.xlabel('Distance metric')
# naming the y axis
plt.ylabel('BER')
plt.legend(title='Dividers') 
# giving a title to my graph
plt.title('Fixed length and baud rate @89.99Kbps')
  
# function to show the plot
plt.show()

In [ ]:
x = [0.09,0.36,0.8,1.44,2.25,3.23]


y10cm = [54.105,54.748,12.337,4.626,48.775,15.317]
y30cm = [5.11,60.125,57.369,2.226,16.331,48.228]
y50cm = [5.99,3.149,3.83,1.745,3.498,2.737]


# plotting the points 

line1, = plt.plot(x,y10cm, label="10cm")
line2, = plt.plot(x,y30cm, label="30cm")
line3, = plt.plot(x,y50cm, label="50cm")

plt.xticks(np.arange(0,4,0.1))
plt.yticks(np.arange(0,100,5))
plt.legend()
plt.ylim(1,100)
plt.xlim(0,5)  
# naming the x axis
plt.xlabel('Distance metric')
# naming the y axis
plt.ylabel('BER')
plt.legend(title='carrier_tag distance') 
# giving a title to my graph
plt.title('Varying distance between tag and carrier')
  
# function to show the plot
plt.show()